# Test enzyme cutting sites in indels that aren't in the wild type

Go to Ape > Edit Current Enzyme Set and copy into supplement/enzyme_list.csv

In [1]:
import pandas as pd
# Read in the file 3_fr5.fastq.csv from Normalized_Matrix where the header is the first row and the index_col is the first column
barcode_indel = pd.read_csv('Normalized_Matrix/F0-FT_R1_001.fastq.csv', header=0, index_col=0)

In [15]:
list(barcode_indel.columns)

['TTCAAGTGCACATCCGAGCGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCGAGGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCGAGGCGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCGAGGGGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCGAGGGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCGAGG',
 'TTCCGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCGAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCCGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCGCGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCGAGGGGCGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCGAGGGACGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACCGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACACGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCGAGGGTCGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCCCAGCCCTACGAGG',
 'TTCAAGTGCACATCCGAGGGCCGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCGAGGGCTACGAGG',
 'TTCAAGTGCACATCCGACGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCCGAAGGCAAGCCCTACGAGG',
 'TTCAAGTGCACATCCGAGGGCCCTACGAGG',
 'TTCAAGTGCACATCCGAGGGCCTACGAGG',
 'TTCAAGTGCACATCCGAGGGAAGCCCTACGAGG',
 'TTC

In [3]:
import pandas as pd
# Read in the file 3_fr5.fastq.csv from Normalized_Matrix
enzymes = pd.read_fwf('./supplement/enzyme_list.csv', header=None)
# Take each row and split it into a list. Only take the first two elements of the list and make a new dataframe
enzymes = enzymes[0].str.split(' ', expand=True)#[[0,1]]
# Take first two columns and rename them to 'Enzyme' and 'Sequence'
enzymes = enzymes[[0,1]].rename(columns={0:'Enzyme', 1:'Sequence'})
# Create a new column that remove only the characters that are not in DNA and not the letter N
# Example: GACNNNN^NNGTC -> GACNNNNNNGTC
# CACCTGC(4/8) -> CACCTGC
enzymes['Sequence Filtered'] = enzymes['Sequence'].str.replace('[^ACGTN]', '')
enzymes

/var/folders/9m/qcg58ft12w72z4h1j7s196w00000gn/T/ipykernel_50824/1342014043.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  enzymes['Sequence Filtered'] = enzymes['Sequence'].str.replace('[^ACGTN]', '')


,Enzyme,Sequence,Sequence Filtered
0,AanI,TTA^TAA,TTATAA
1,AarI,CACCTGC(4/8),CACCTGC
2,AasI,GACNNNN^NNGTC,GACNNNNNNGTC
3,AatII,GACGT^C,GACGTC
4,AbsI,CC^TCGAGG,CCTCGAGG
...,...,...,...
605,XmnI,GAANN^NNTTC,GAANNNNTTC
606,XspI,C^TAG,CTAG
607,ZraI,GAC^GTC,GACGTC
608,ZrmI,AGT^ACT,AGTACT


In [ ]:
# Find where the Enzyme column is ApaLI
enzymes[enzymes['Enzyme'] == 'ApaLI']

,Enzyme,Sequence,Sequence Filtered
48,ApaLI,G^TGCAC,GTGCAC


In [42]:
# Create a new dataframe with the same index and dimensions as barcode_indel
# Go through each row in barcode_indel and check if the indel contains any of the sequences in the enzyme list
# If it does, then add the enzyme name to that index in the new dataframe
# If it does not, then add 'None' to that index in the new dataframe
# function for reverse complement
def reverse_complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    bases = list(seq)
    bases = [complement[base] for base in bases]
    return ''.join(bases)[::-1]
enzyme_indel = pd.DataFrame(index=barcode_indel.index, columns=barcode_indel.columns)
# Turn enzyme_indel into only the first row
enzyme_indel = enzyme_indel.iloc[0:1]

for i in range(len(enzyme_indel.index)): # only the first row
    for j in range(len(enzyme_indel.columns)):
        for k in range(len(enzymes)):
            enzyme_seq = enzymes['Sequence Filtered'][k]
            # Construct a grep using enzyme_seq where N is replaced with .*
            # If the indel contains the enzyme sequence, then add the enzyme name to the new dataframe as a list
            # If there is already a list, then add the enzyme name to the list
            # If the indel does not contain the enzyme sequence, then add 'None' to the new dataframe
            pattern_normal = enzyme_seq.replace('N', '.*')
            # Replace W with A or T, S with C or G, M with A or C, K with G or T, R with A or G, Y with C or T
            pattern_normal = pattern_normal.replace('W', '[AT]').replace('S', '[CG]').replace('M', '[AC]').replace('K', '[GT]').replace('R', '[AG]').replace('Y', '[CT]')
            pattern_reverse = reverse_complement(enzyme_seq).replace('N', '.*').replace('W', '[AT]').replace('S', '[CG]').replace('M', '[AC]').replace('K', '[GT]').replace('R', '[AG]').replace('Y', '[CT]')
            if barcode_indel.columns[j].find(pattern_normal) != -1 or barcode_indel.columns[j].find(pattern_reverse) != -1:
                enzyme_indel.iloc[i,j] = [enzymes['Enzyme'][k]] if not isinstance(enzyme_indel.iloc[i,j], list) else enzyme_indel.iloc[i,j]+ [enzymes['Enzyme'][k]]
            # Do print statement to see progress
enzyme_indel

,TTCAAGTGCACATCCGAGCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGGGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGG,TTCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCGAAGGCAAGCCCTACGAGG,TTCAAGTGCGAAGGCAAGCCCTACGAGG,...,TACAAGTGCACATCCGAGGGGCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGGCCGAAGGCAAGCCCGACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCGACGAGG,TTCAAGTGCACATCCCGATCAACGCATCCGGAGTTGACGCCAAAGCAATCCTGAGCGCTAGGGGGGAGGTGTGGGAGGTTTTTTAAAGCAAGTAAAACCTCTACGAAGGCAAGCCCTACGAGG,TTCAAGTGCACCCGAAGGCAAGCCCTCCGAGG,TTCAAGCCCACGAAGGCAAGCCCTACGAGG,TACAAGTGCACATCCGAGCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACCGAAGGCATGCCCTACGAGG,TTCAAGGGCACATCCGAGGGGCCCTACGAGG,TTCAAGTGCAAATCCGAGCGAAGGCAAGCCCTACGAGG
Barcode,,,,,,,,,,,,,,,,,,,,,
TTCCG,"[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[CviJI, CviKI-1, HpyAV, MnlI]","[Alw44I, ApaLI, CviJI, CviKI-1, FaiI, HpyAV, H...","[CviJI, CviKI-1, HpyAV, MnlI]",...,"[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[AccB1I, AcoI, Alw21I, Alw44I, ApaLI, AvaII, B...","[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[AccIII, AcyI, AfeI, AgsI, AjnI, Alw44I, Ama87...","[Alw44I, ApaLI, CviJI, CviKI-1, HpyAV, HpyCH4V...","[CviJI, CviKI-1, HpyAV, MnlI]","[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[Alw44I, ApaLI, BstNSI, CviAII, CviJI, CviKI-1...","[AccB1I, AcoI, Alw21I, ApaI, AvaII, BaeGI, Ban...","[CviJI, CviKI-1, FaiI, HpyAV, HpyCH4V, MnlI, S..."


In [84]:
enzyme_indel

,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGGCCGATTTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGCGAAGGCAAGCCCTACGAGGCCGATTTCAAGTGCACATCCGAGCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGGGAAGGCAAGCCCTACGAGGCGGAATTCAAGTGCACATCCGAGGGGAAGGCAAGCCCTACGAGG,TTCAAGTGCACACGAAGGCAAGCCCTACGAGGTTGAATTCAAGTGCACACGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGGCGGAATTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGCGAAGGCAAGCCCTACGAGGCGGAATTCAAGTGCACATCCGAGCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGGGAAGGCAAGCCCTACGAGGCCGATTTCAAGTGCACATCCGAGGGGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGGCAAGCCCTACGAGGCCGATTTCAAGTGCACATCCGAGGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGGCAAGCCCTACGAGGCGGAATTCAAGTGCACATCCGAGGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGGTAGCATTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,...,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGGTCTAATTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGGTCGTTTTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGGTCGTATTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGGTCGGTTTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGGTCGCATTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGGTCGATTTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGGTCCGTTTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGGTCCCATTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGGTCCAGTTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,TTTTCTACGAGGCCGATTTTTCTACGAGG
Barcode,,,,,,,,,,,,,,,,,,,,,
ATCGG,"[AccB1I, AcoI, Alw21I, Alw44I, ApaLI, AvaII, B...","[AccB1I, AcoI, Alw21I, Alw44I, ApaLI, AvaII, B...","[AciI, AcsI, Alw44I, ApaLI, ApoI, BseGI, BspAC...","[AcsI, Alw44I, ApaLI, ApoI, CviJI, CviKI-1, Ec...","[AciI, AcsI, Alw44I, ApaLI, ApoI, BseGI, BspAC...","[AciI, AcsI, Alw44I, ApaLI, ApoI, BseGI, BspAC...","[AccB1I, AcoI, Alw21I, Alw44I, ApaLI, AvaII, B...","[AccB1I, AcoI, Alw21I, Alw44I, ApaLI, AvaII, B...","[AciI, AcsI, Alw44I, ApaLI, ApoI, BseGI, BspAC...","[Alw44I, ApaLI, BseGI, BsmI, BstF5I, BtsCI, Cv...",...,"[AcsI, Alw44I, ApaLI, ApoI, BseGI, BstF5I, Bts...","[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[Alw44I, ApaLI, BseGI, BsmI, BstF5I, BtsCI, Cv...","[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[Alw44I, ApaLI, BseGI, BstF5I, BtsCI, CviJI, C...","[Alw44I, ApaLI, BseGI, BslFI, BsmFI, BstF5I, B...","[Alw44I, ApaLI, Bse1I, BseGI, BseNI, BsrI, Bst...","[AccB1I, AcoI, Alw21I, AvaII, BaeGI, BanI, Ban..."


In [27]:
# Read in the df from Step6_Indel/F0-FT_R1_001.fastq.csv
df = pd.read_csv('Step6_Indel/F0-FT_R1_001.fastq', header=None)
# Value counts and find the most common sequence
df[0].value_counts().head(1)

CCTCGTAGGGCTTGCCTTCGCCCTCGGATGTGCACTTGAA    119647
Name: 0, dtype: int64

In [29]:
reverse_complement('CCTCGTAGGGCTTGCCTTCGCCCTCGGATGTGCACTTGAA')

'TTCAAGTGCACATCCGAGGGCGAAGGCAAGCCCTACGAGG'

In [43]:
# Read in the df from Step6_Indel/F0-FT_R1_001.fastq.csv
df = pd.read_csv('Step6_Indel/F0-FT_R1_001.fastq', header=None)
# Value counts and find the most common sequence
wild_type = df[0].value_counts().head(1)
# Reverse complement the most common sequence
wild_type = reverse_complement(wild_type.index[0])
# Show the value counts without the most common sequence
# Construct a grep using wild_type where N is replaced with .*
# If the indel contains the enzyme sequence, then add the enzyme name to the new dataframe as a list
# If there is already a list, then add the enzyme name to the list
# If the indel does not contain the enzyme sequence, then add 'None' to the new dataframe
wild_type_enzymes = []
for k in range(len(enzymes)):
    enzyme_seq = enzymes['Sequence Filtered'][k]
    # print(enzyme_seq)
    # Construct a grep using enzyme_seq where N is replaced with .*
    # If the indel contains the enzyme sequence, then add the enzyme name to the new dataframe as a list
    # If there is already a list, then add the enzyme name to the list
    # If the indel does not contain the enzyme sequence, then add 'None' to the new dataframe
    pattern_normal = enzyme_seq.replace('N', '.*')
    pattern_normal = pattern_normal.replace('W', '[AT]').replace('S', '[CG]').replace('M', '[AC]').replace('K', '[GT]').replace('R', '[AG]').replace('Y', '[CT]')
    pattern_reverse = reverse_complement(enzyme_seq).replace('N', '.*').replace('W', '[AT]').replace('S', '[CG]').replace('M', '[AC]').replace('K', '[GT]').replace('R', '[AG]').replace('Y', '[CT]')
    # if the enzyme sequence is A
    if wild_type.find(pattern_normal) != -1 or wild_type.find(pattern_reverse) != -1:
        wild_type_enzymes += [enzymes['Enzyme'][k]]
wild_type_enzymes
# Go through all the indices in enzyme_indel. If the array at the index includes any of the enzymes in wild_type_enzymes, then remove it.
# If the array is empty, then don't do anything
for i in range(len(enzyme_indel.index)):
    for j in range(len(enzyme_indel.columns)):
        if isinstance(enzyme_indel.iloc[i,j], list):
            enzyme_indel.iloc[i,j] = [x for x in enzyme_indel.iloc[i,j] if x not in wild_type_enzymes]
        else:
            continue
enzyme_indel

,TTCAAGTGCACATCCGAGCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGGGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGG,TTCCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCGAAGGCAAGCCCTACGAGG,TTCAAGTGCGAAGGCAAGCCCTACGAGG,...,TACAAGTGCACATCCGAGGGGCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACATCCGAGGGCCGAAGGCAAGCCCGACGAGG,TTCAAGTGCACATCCGAAGGCAAGCCCGACGAGG,TTCAAGTGCACATCCCGATCAACGCATCCGGAGTTGACGCCAAAGCAATCCTGAGCGCTAGGGGGGAGGTGTGGGAGGTTTTTTAAAGCAAGTAAAACCTCTACGAAGGCAAGCCCTACGAGG,TTCAAGTGCACCCGAAGGCAAGCCCTCCGAGG,TTCAAGCCCACGAAGGCAAGCCCTACGAGG,TACAAGTGCACATCCGAGCGAAGGCAAGCCCTACGAGG,TTCAAGTGCACCGAAGGCATGCCCTACGAGG,TTCAAGGGCACATCCGAGGGGCCCTACGAGG,TTCAAGTGCAAATCCGAGCGAAGGCAAGCCCTACGAGG
Barcode,,,,,,,,,,,,,,,,,,,,,
TTCCG,[],[],[],[],[],[],[],[],[TaqI],[],...,[],"[AccB1I, AcoI, Alw21I, AvaII, BaeGI, BanI, Ban...",[],"[AccIII, AcyI, AfeI, AgsI, AjnI, Ama87I, Aor13...",[],[],[],"[BstNSI, CviAII, FaeI, FatI, Hin1II, Hsp92II, ...","[AccB1I, AcoI, Alw21I, ApaI, AvaII, BaeGI, Ban...",[]


In [49]:
enzyme_indel['TTCAAGTGCACCGAAGGCATGCCCTACGAGG'][0]

['BstNSI',
 'CviAII',
 'FaeI',
 'FatI',
 'Hin1II',
 'Hsp92II',
 'NlaIII',
 'NspI',
 'PaeI',
 'SphI',
 'XceI']

In [52]:
enzymes[enzymes['Enzyme'] == 'FatI']

,Enzyme,Sequence,Sequence Filtered
339,FatI,^CATG,CATG


In [23]:
reverse_complement(wild_type)

'CCTCGTAGGGCTTGCCTTCGCCCTCGGATGTGCACTTGAA'

In [21]:
wild_type_enzymes

['CviJI', 'CviKI-1', 'FaiI', 'HpyAV', 'HpyCH4V', 'MnlI', 'SetI']